<a href="https://colab.research.google.com/github/Dee-Nwanjah/Python-Pandas-Mastery-Through-Real-Data/blob/main/7.)COVID_Explorer(MAIN-PROJECT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import streamlit as st
from datetime import datetime, timedelta
import requests
import json

# Create notebook: "Day7_COVID_Explorer"
print("COVID-19 Data Explorer - Portfolio Project")
print("="*50)

# Function to download COVID data
def download_covid_data():
    """Download latest COVID-19 data from reliable sources"""

    # Primary source: Our World in Data
    url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"

    try:
        print("Downloading COVID-19 data...")
        df = pd.read_csv(url)
        print(f"✅ Successfully downloaded {len(df)} records")
        return df
    except Exception as e:
        print(f"❌ Error downloading data: {e}")
        # Fallback: create sample data for demonstration
        return create_sample_covid_data()

def create_sample_covid_data():
    """Create sample COVID data if download fails"""
    print("Creating sample data for demonstration...")

    countries = ['United States', 'United Kingdom', 'Germany', 'France', 'Italy', 'Spain', 'Canada', 'Australia']
    dates = pd.date_range('2020-01-01', '2023-12-31', freq='D')

    data = []
    for country in countries:
        for date in dates:
            # Simulate realistic COVID trends
            days_since_start = (date - dates[0]).days
            base_cases = max(0, np.random.poisson(100) * (1 + 0.01 * days_since_start))

            data.append({
                'location': country,
                'date': date,
                'new_cases': max(0, base_cases + np.random.normal(0, 50)),
                'total_cases': None,  # Will calculate cumulative
                'new_deaths': max(0, np.random.poisson(2)),
                'total_deaths': None,  # Will calculate cumulative
                'population': np.random.randint(10000000, 350000000)
            })

    df = pd.DataFrame(data)

    # Calculate cumulative totals
    for country in countries:
        mask = df['location'] == country
        df.loc[mask, 'total_cases'] = df.loc[mask, 'new_cases'].cumsum()
        df.loc[mask, 'total_deaths'] = df.loc[mask, 'new_deaths'].cumsum()

    return df

# Download data
covid_df = download_covid_data()
covid_df['date'] = pd.to_datetime(covid_df['date'])

# Basic data exploration
print(f"\nDataset shape: {covid_df.shape}")
print(f"Date range: {covid_df['date'].min()} to {covid_df['date'].max()}")
print(f"Countries: {covid_df['location'].nunique()}")
print(f"Columns: {list(covid_df.columns)}")

COVID-19 Data Explorer - Portfolio Project
✅ Successfully downloaded 429435 records

Dataset shape: (429435, 67)
Date range: 2020-01-01 00:00:00 to 2024-08-14 00:00:00
Countries: 255
Columns: ['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients', 'icu_patients_per_million', 'hosp_patients', 'hosp_patients_per_million', 'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations', 'people_vaccinated', 'people_fully_v

Data Cleaning and Preparation

In [3]:
# Clean and prepare the data
def clean_covid_data(df):
    """Clean and prepare COVID data for analysis"""

    # Filter for countries only (remove continents and world totals)
    exclude_locations = ['World', 'Europe', 'Asia', 'North America', 'South America', 'Africa', 'Oceania',
                        'European Union', 'High income', 'Upper middle income', 'Lower middle income', 'Low income']

    df_clean = df[~df['location'].isin(exclude_locations)].copy()

    # Fill missing values for key columns
    numeric_columns = ['new_cases', 'new_deaths', 'total_cases', 'total_deaths', 'population']
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0)

    # Add calculated fields
    df_clean['cases_per_million'] = (df_clean['total_cases'] / df_clean['population'] * 1000000).fillna(0)
    df_clean['deaths_per_million'] = (df_clean['total_deaths'] / df_clean['population'] * 1000000).fillna(0)

    # Add time-based columns
    df_clean['year'] = df_clean['date'].dt.year
    df_clean['month'] = df_clean['date'].dt.month
    df_clean['week'] = df_clean['date'].dt.isocalendar().week

    # Calculate 7-day rolling averages
    df_clean = df_clean.sort_values(['location', 'date'])
    df_clean['new_cases_7day_avg'] = df_clean.groupby('location')['new_cases'].rolling(7, min_periods=1).mean().reset_index(0, drop=True)
    df_clean['new_deaths_7day_avg'] = df_clean.groupby('location')['new_deaths'].rolling(7, min_periods=1).mean().reset_index(0, drop=True)

    return df_clean

covid_clean = clean_covid_data(covid_df)

print("Data cleaning complete!")
print(f"Cleaned dataset shape: {covid_clean.shape}")
print(f"Top 10 countries by total cases:")
top_countries = covid_clean.groupby('location')['total_cases'].max().sort_values(ascending=False).head(10)
print(top_countries)

Data cleaning complete!
Cleaned dataset shape: (417680, 74)
Top 10 countries by total cases:
location
High-income countries            429044049.0
Upper-middle-income countries    251753518.0
European Union (27)              185822587.0
United States                    103436829.0
China                             99373219.0
Lower-middle-income countries     91954400.0
India                             45041748.0
France                            38997490.0
Germany                           38437756.0
Brazil                            37511921.0
Name: total_cases, dtype: float64


Interactive Dashboard

In [9]:
# Create interactive visualizations using Plotly
def create_covid_dashboard(df):
    """Create comprehensive COVID-19 dashboard using Plotly Express"""

    # Dashboard 1: Global Overview
    # Top 15 countries by total cases
    top_15 = df.groupby('location')['total_cases'].max().sort_values(ascending=False).head(15).reset_index()
    fig1_bar = px.bar(top_15, x='total_cases', y='location', orientation='h', title='Top 15 Countries by Total Cases', color='location')

    # Global daily trend
    global_daily = df.groupby('date')['new_cases'].sum().reset_index()
    fig1_line = px.line(global_daily, x='date', y='new_cases', title='Global Daily New Cases Trend')

    # Cases per Million Choropleth Map (Latest Data)
    latest_data = df.loc[df.groupby('location')['date'].idxmax()].copy()
    fig1_map = px.choropleth(
        latest_data,
        locations='location',
        locationmode='country names',
        color='cases_per_million',
        hover_name='location',
        color_continuous_scale='Plasma',
        title='Global Cases per Million'
    )

    # Deaths vs Cases Correlation (Latest Data)
    fig1_scatter = px.scatter(
        latest_data,
        x='total_cases',
        y='total_deaths',
        size='population',
        color='location',
        hover_name='location',
        title='Deaths vs Cases Correlation (Latest Data)'
    )

    # Combine figures into a dashboard layout using make_subplots for better control over positioning
    fig1 = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Top 15 Countries by Total Cases', 'Global Daily New Cases Trend',
                       'Global Cases per Million', 'Deaths vs Cases Correlation'),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "choropleth"}, {"type": "scatter"}]]
    )

    # Add traces from Plotly Express figures to the subplot
    for data in fig1_bar.data:
        fig1.add_trace(data, row=1, col=1)
    for data in fig1_line.data:
        fig1.add_trace(data, row=1, col=2)
    for data in fig1_map.data:
        fig1.add_trace(data, row=2, col=1)
    for data in fig1_scatter.data:
        fig1.add_trace(data, row=2, col=2)


    fig1.update_layout(height=800, showlegend=False, title_text="COVID-19 Global Dashboard")

    return fig1

def create_country_comparison(df, countries=['United States', 'United Kingdom', 'Germany', 'France', 'Italy']):
    """Create country comparison charts using Plotly Express"""

    # Filter for selected countries
    country_data = df[df['location'].isin(countries)].copy()

    # Create subplots using Plotly Express for easier creation of individual charts
    fig_cases_avg = px.line(country_data, x='date', y='new_cases_7day_avg', color='location', title='Daily New Cases (7-day average)')
    fig_cases_cum = px.line(country_data, x='date', y='total_cases', color='location', title='Cumulative Cases')
    fig_deaths_avg = px.line(country_data, x='date', y='new_deaths_7day_avg', color='location', title='Daily Deaths (7-day average)')
    fig_cases_per_million = px.line(country_data, x='date', y='cases_per_million', color='location', title='Cases per Million Population')


    # Combine figures into a dashboard layout using make_subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Daily New Cases (7-day average)', 'Cumulative Cases',
                       'Daily Deaths (7-day average)', 'Cases per Million Population'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Add traces from Plotly Express figures to the subplot
    for data in fig_cases_avg.data:
        fig.add_trace(data, row=1, col=1)
    for data in fig_cases_cum.data:
        fig.add_trace(data, row=1, col=2)
    for data in fig_deaths_avg.data:
        fig.add_trace(data, row=2, col=1)
    for data in fig_cases_per_million.data:
        fig.add_trace(data, row=2, col=2)

    fig.update_layout(height=800, title_text="Country Comparison Dashboard")


    return fig

# Create dashboards
print("Creating interactive dashboards...")
dashboard1 = create_covid_dashboard(covid_clean)
dashboard2 = create_country_comparison(covid_clean)

# Display dashboards (in Jupyter notebook)
dashboard1.show()
dashboard2.show()

print("✅ Interactive dashboards created!")

Creating interactive dashboards...


✅ Interactive dashboards created!


Building Streamlit Web Application

In [ ]:
# Create Streamlit web app file
streamlit_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime, timedelta

# Configure page
st.set_page_config(
    page_title="COVID-19 Data Explorer",
    page_icon="🦠",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Load data
@st.cache_data
def load_covid_data():
    """Load and cache COVID-19 data"""
    try:
        url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
        df = pd.read_csv(url)
        df['date'] = pd.to_datetime(df['date'])

        # Clean data
        exclude_locations = ['World', 'Europe', 'Asia', 'North America', 'South America', 'Africa', 'Oceania']
        df = df[~df['location'].isin(exclude_locations)]

        # Fill missing values
        numeric_columns = ['new_cases', 'new_deaths', 'total_cases', 'total_deaths', 'population']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

        # Add calculated fields
        df['cases_per_million'] = (df['total_cases'] / df['population'] * 1000000).fillna(0)
        df['deaths_per_million'] = (df['total_deaths'] / df['population'] * 1000000).fillna(0)

        return df
    except:
        st.error("Failed to load data")
        return None

# Main app
def main():
    st.title("🦠 COVID-19 Data Explorer")
    st.markdown("Interactive analysis of global COVID-19 trends and patterns")

    # Load data
    df = load_covid_data()
    if df is None:
        return

    # Sidebar controls
    st.sidebar.header("🎛️ Controls")

    # Date range selector
    min_date = df['date'].min()
    max_date = df['date'].max()

    date_range = st.sidebar.date_input(
        "Select Date Range",
        value=(min_date, max_date),
        min_value=min_date,
        max_value=max_date
    )

    # Country selector
    countries = sorted(df['location'].unique())
    selected_countries = st.sidebar.multiselect(
        "Select Countries",
        countries,
        default=['United States', 'United Kingdom', 'Germany', 'France', 'Italy']
    )

    # Metric selector
    metric = st.sidebar.selectbox(
        "Select Metric",
        ['total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'cases_per_million', 'deaths_per_million']
    )

    # Filter data
    if len(date_range) == 2:
        filtered_df = df[
            (df['date'] >= pd.to_datetime(date_range[0])) &
            (df['date'] <= pd.to_datetime(date_range[1]))
        ]
    else:
        filtered_df = df

    # Main dashboard
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        total_cases = filtered_df['total_cases'].max()
        st.metric("Global Cases", f"{total_cases:,.0f}")

    with col2:
        total_deaths = filtered_df['total_deaths'].max()
        st.metric("Global Deaths", f"{total_deaths:,.0f}")

    with col3:
        countries_affected = filtered_df['location'].nunique()
        st.metric("Countries Affected", f"{countries_affected:,}")

    with col4:
        latest_date = filtered_df['date'].max()
        st.metric("Data Updated", latest_date.strftime('%Y-%m-%d'))

    # Time series chart
    st.header("📈 Time Series Analysis")

    if selected_countries:
        country_data = filtered_df[filtered_df['location'].isin(selected_countries)]

        fig = px.line(
            country_data,
            x='date',
            y=metric,
            color='location',
            title=f'{metric.replace("_", " ").title()} Over Time',
            template='plotly_white'
        )

        fig.update_layout(
            height=500,
            xaxis_title="Date",
            yaxis_title=metric.replace("_", " ").title()
        )

        st.plotly_chart(fig, use_container_width=True)

    # Country comparison
    st.header("🌍 Country Comparison")

    col1, col2 = st.columns(2)

    with col1:
        # Top countries bar chart
        top_countries = filtered_df.groupby('location')[metric].max().sort_values(ascending=False).head(15)

        fig_bar = px.bar(
            x=top_countries.values,
            y=top_countries.index,
            orientation='h',
            title=f'Top 15 Countries by {metric.replace("_", " ").title()}',
            template='plotly_white'
        )

        fig_bar.update_layout(height=400)
        st.plotly_chart(fig_bar, use_container_width=True)

    with col2:
        # Scatter plot for correlation
        if len(selected_countries) > 0:
            scatter_data = filtered_df[
                (filtered_df['location'].isin(selected_countries)) &
                (filtered_df['date'] == filtered_df['date'].max())
            ]

            fig_scatter = px.scatter(
                scatter_data,
                x='total_cases',
                y='total_deaths',
                color='location',
                size='population',
                hover_name='location',
                title='Cases vs Deaths (Latest Data)',
                template='plotly_white'
            )

            fig_scatter.update_layout(height=400)
            st.plotly_chart(fig_scatter, use_container_width=True)

    # World map
    st.header("🗺️ Global Heat Map")

    # Get latest data for each country
    latest_data = filtered_df.loc[filtered_df.groupby('location')['date'].idxmax()]

    fig_map = px.choropleth(
        latest_data,
        locations='location',
        locationmode='country names',
        color=metric,
        hover_name='location',
        color_continuous_scale='Reds',
        title=f'Global {metric.replace("_", " ").title()} Distribution'
    )

    fig_map.update_layout(height=500)
    st.plotly_chart(fig_map, use_container_width=True)

    # Data table
    st.header("📊 Data Table")

    if selected_countries:
        table_data = country_data[
            ['location', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'cases_per_million']
        ].sort_values(['location', 'date'], ascending=[True, False])

        st.dataframe(table_data, use_container_width=True, height=400)

    # Download section
    st.header("💾 Download Data")

    if selected_countries:
        csv = country_data.to_csv(index=False)
        st.download_button(
            label="Download Selected Data as CSV",
            data=csv,
            file_name=f'covid_data_{datetime.now().strftime("%Y%m%d")}.csv',
            mime='text/csv'
        )

if __name__ == "__main__":
    main()
'''

# Save Streamlit app to file
with open('covid_explorer_app.py', 'w') as f:
    f.write(streamlit_code)

print("✅ Streamlit app saved as 'covid_explorer_app.py'")
print("\n🚀 To run the app:")
print("1. Open terminal/command prompt")
print("2. Navigate to your project folder")
print("3. Run: streamlit run covid_explorer_app.py")
print("4. App will open in your browser automatically!")

✅ Streamlit app saved as 'covid_explorer_app.py'

🚀 To run the app:
1. Open terminal/command prompt
2. Navigate to your project folder
3. Run: streamlit run covid_explorer_app.py
4. App will open in your browser automatically!


In [ ]:
!streamlit run covid_explorer_app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.151.182:8501

  Stopping...
  Stopping...


Advanced Analytics

In [8]:
# Add advanced analytics functions
def analyze_covid_trends(df):
    """Perform advanced trend analysis"""

    # Calculate growth rates
    df_sorted = df.sort_values(['location', 'date']).copy() # Add .copy() to avoid SettingWithCopyWarning
    df_sorted['cases_growth_rate'] = df_sorted.groupby('location')['total_cases'].pct_change() * 100
    df_sorted['deaths_growth_rate'] = df_sorted.groupby('location')['total_deaths'].pct_change() * 100

    # Identify peaks and waves
    df_sorted['cases_7day_avg'] = df_sorted.groupby('location')['new_cases'].rolling(7, min_periods=1).mean().reset_index(0, drop=True)

    # Calculate case fatality rate, handling division by zero
    df_sorted['case_fatality_rate'] = (df_sorted['total_deaths'] / df_sorted['total_cases'] * 100)
    df_sorted['case_fatality_rate'] = df_sorted['case_fatality_rate'].replace([np.inf, -np.inf], np.nan).fillna(0) # Handle potential inf and NaN

    return df_sorted

def generate_insights(df):
    """Generate automated insights from the data"""

    insights = []

    # Latest global statistics
    latest_date = df['date'].max()
    latest_data = df[df['date'] == latest_date].copy() # Add .copy()

    total_global_cases = latest_data['total_cases'].sum()
    total_global_deaths = latest_data['total_deaths'].sum()
    # Handle potential division by zero for global CFR
    global_cfr = (total_global_deaths / total_global_cases) * 100 if total_global_cases > 0 else 0

    insights.append(f"📊 As of {latest_date.strftime('%B %d, %Y')}, there have been {total_global_cases:,.0f} confirmed cases globally")
    insights.append(f"💔 Global deaths: {total_global_deaths:,.0f} (Case Fatality Rate: {global_cfr:.2f}%)")

    # Most affected countries
    top_cases = latest_data.nlargest(5, 'total_cases')['location'].tolist()
    top_deaths = latest_data.nlargest(5, 'total_deaths')['location'].tolist()

    insights.append(f"🏆 Countries with most cases: {', '.join(top_cases)}")
    insights.append(f"💀 Countries with most deaths: {', '.join(top_deaths)}")

    # Recent trends
    last_7_days = df[df['date'] >= (latest_date - timedelta(days=7))].copy() # Add .copy()
    recent_cases = last_7_days.groupby('date')['new_cases'].sum()

    if len(recent_cases) > 1:
        trend = "increasing" if recent_cases.iloc[-1] > recent_cases.iloc[0] else "decreasing"
        insights.append(f"📈 Global daily cases are {trend} over the past week")

    # Cases per million analysis
    # Ensure 'population' column exists and is numeric before calculation
    if 'population' in latest_data.columns:
        latest_data['cases_per_million'] = (latest_data['total_cases'] / latest_data['population'] * 1000000).fillna(0)
        latest_data_filtered = latest_data[latest_data['location'].isin(top_countries)].copy() # Filter for top countries
        if not latest_data_filtered.empty:
            highest_per_capita = latest_data_filtered.nlargest(3, 'cases_per_million')[['location', 'cases_per_million']]
            insights.append(f"📍 Highest cases per million among top countries: {highest_per_capita.iloc[0]['location']} ({highest_per_capita.iloc[0]['cases_per_million']:,.0f} per million)")
        else:
            insights.append("⚠️ Cannot calculate cases per million for top countries: data not available.")

    else:
        insights.append("⚠️ Cannot calculate cases per million: 'population' column not found.")


    return insights

# Perform advanced analysis
covid_advanced = analyze_covid_trends(covid_clean)
insights = generate_insights(covid_advanced)

print("=== COVID-19 DATA INSIGHTS ===")
for insight in insights:
    print(insight)

# Create advanced visualization
def create_advanced_charts(df):
    """Create advanced analytical charts"""

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Growth Rate Trends', 'Case Fatality Rate by Country',
                       'Wave Pattern Analysis', 'Cases vs Population'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Select top 5 countries for analysis
    # Ensure 'total_cases' is not all zeros before selecting top countries
    if df['total_cases'].sum() > 0:
        top_countries = df.groupby('location')['total_cases'].max().nlargest(5).index
    else:
        top_countries = [] # Handle case where there are no cases

    colors = px.colors.qualitative.Set1

    for i, country in enumerate(top_countries):
        country_data = df[df['location'] == country].copy() # Add .copy()

        # Growth rate trends
        fig.add_trace(
            go.Scatter(
                x=country_data['date'],
                y=country_data['cases_growth_rate'].rolling(7).mean(),
                mode='lines',
                name=country,
                line=dict(color=colors[i])
            ),
            row=1, col=1
        )

        # Case fatality rate
        if not country_data.empty: # Check if country_data is not empty
            latest_cfr = country_data['case_fatality_rate'].iloc[-1]
            fig.add_trace(
                go.Bar(
                    x=[country],
                    y=[latest_cfr],
                    name=country,
                    marker_color=colors[i],
                    showlegend=False
                ),
                row=1, col=2
            )

        # Wave analysis (7-day average)
        fig.add_trace(
            go.Scatter(
                x=country_data['date'],
                y=country_data['cases_7day_avg'],
                mode='lines',
                name=country,
                line=dict(color=colors[i]),
                showlegend=False
            ),
            row=2, col=1
        )

    # Population vs cases scatter
    latest_pop_data = df[df['date'] == df['date'].max()].copy() # Add .copy()
    # Ensure 'population' and 'total_cases' columns exist and are numeric
    if 'population' in latest_pop_data.columns and 'total_cases' in latest_pop_data.columns:
         fig.add_trace(
            go.Scatter(
                x=latest_pop_data['population'],
                y=latest_pop_data['total_cases'],
                mode='markers',
                text=latest_pop_data['location'],
                marker=dict(size=8, color=latest_pop_data['location'].astype('category').cat.codes, colorscale='Viridis', showscale=True), # Map countries to colors
                showlegend=False
            ),
            row=2, col=2
        )


    fig.update_layout(height=800, title_text="Advanced COVID-19 Analytics")

    return fig

advanced_chart = create_advanced_charts(covid_advanced)
advanced_chart.show()

print("✅ Advanced analytics complete!")

=== COVID-19 DATA INSIGHTS ===
📊 As of August 14, 2024, there have been 0 confirmed cases globally
💔 Global deaths: 0 (Case Fatality Rate: 0.00%)
🏆 Countries with most cases: European Union (27), High-income countries, Lithuania, Malaysia, Upper-middle-income countries
💀 Countries with most deaths: European Union (27), High-income countries, Lithuania, Malaysia, Upper-middle-income countries
📈 Global daily cases are decreasing over the past week
⚠️ Cannot calculate cases per million for top countries: data not available.


✅ Advanced analytics complete!


## Summary:

### Data Analysis Key Findings

*   The analysis process involved downloading, cleaning, and preparing COVID-19 data.
*   Interactive dashboards were created using Plotly and Plotly Express.
*   The key data structures used were pandas DataFrames: `covid_df` (raw data), `covid_clean` (cleaned data), and `covid_advanced` (data for advanced analysis).
*   Deployment instructions for a Streamlit application were outlined, including creating a `requirements.txt` file and using the command `streamlit run <app_file_name>.py`.

### Insights

*   The documentation provides a clear guide for understanding the code and the data analysis process.
*   The outlined deployment steps offer a straightforward path to making the interactive dashboards accessible via a web application.
